### Fetching Data

##### This notebook will fetch data used to build the Olympics dashboard.
- Medal counts (live)
- Athletes per country
- GDP
- Population

---

### 1. Medal counts

In [7]:
import requests
import pandas as pd

In [18]:
url = 'https://web-cdn.api.bbci.co.uk/wc-poll-data/container/sport-olympics-medals?tournament=paris-2024'

headers = {
    'accept': 'application/json',
    'accept-language': 'en-GB,en;q=0.7',
    # 'if-none-match': '"e0086ea4e830e3bcc82cc414da1b34931a3c4466"',    # add this header to get 304 response, likely corresponds to our web session
    'origin': 'https://www.bbc.co.uk',
    'priority': 'u=1, i',
    'referer': 'https://www.bbc.co.uk/',
    'sec-ch-ua': '"Brave";v="125", "Chromium";v="125", "Not.A/Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'cross-site',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'
}

response = requests.get(url, headers=headers)

print(response.json())


{'standing': [{'country': {'code': 'JPN', 'name': 'Japan', 'urn': 'urn:bbc:sportsdata:olympics:country:japan'}, 'rank': 1, 'sequence': 1, 'medals': {'gold': 6, 'silver': 2, 'total': 12, 'bronze': 4}}, {'country': {'code': 'FRA', 'name': 'France', 'urn': 'urn:bbc:sportsdata:olympics:country:france'}, 'rank': 2, 'sequence': 2, 'medals': {'gold': 5, 'silver': 8, 'total': 16, 'bronze': 3}}, {'country': {'code': 'CHN', 'name': 'China', 'urn': 'urn:bbc:sportsdata:olympics:country:china'}, 'rank': 3, 'sequence': 3, 'medals': {'gold': 5, 'silver': 5, 'total': 12, 'bronze': 2}}, {'country': {'code': 'AUS', 'name': 'Australia', 'urn': 'urn:bbc:sportsdata:olympics:country:australia'}, 'rank': 4, 'sequence': 4, 'medals': {'gold': 5, 'silver': 4, 'total': 9, 'bronze': 0}}, {'country': {'code': 'KOR', 'name': 'South Korea', 'urn': 'urn:bbc:sportsdata:olympics:country:south-korea'}, 'rank': 5, 'sequence': 5, 'medals': {'gold': 5, 'silver': 3, 'total': 9, 'bronze': 1}}, {'country': {'code': 'US', 'nam

In [19]:
response.status_code

200

In [15]:
df = pd.DataFrame(response.json()['standing'])

# Expand the 'country' column into separate columns
df = pd.concat([df.drop(['country', 'medals'], axis=1), df['country'].apply(pd.Series), df['medals'].apply(pd.Series)], axis=1).drop(columns=['urn'])
df

,rank,sequence,code,name,gold,silver,total,bronze
0,1,1,CHN,China,4,2,8,2
1,2,2,JPN,Japan,4,2,7,1
2,2,3,KOR,South Korea,4,2,7,1
3,4,4,AUS,Australia,4,2,6,0
4,5,5,US,United States,3,6,12,3
...,...,...,...,...,...,...,...,...
200,30,201,BVI,British Virgin Islands,0,0,0,0
201,30,202,AVI,American Virgin Islands,0,0,0,0
202,30,203,YEM,Yemen,0,0,0,0
203,30,204,ZAM,Zambia,0,0,0,0


In [22]:
s3_url ='https://eco-data-cache.s3.eu-west-2.amazonaws.com/olympics/live_results/medals.csv'
# Create dataframe using csv from s3 url
df = pd.read_csv(s3_url)
df

,rank,sequence,code,name,gold,silver,total,bronze
0,1,1,JPN,Japan,6,2,12,4
1,2,2,FRA,France,5,8,16,3
2,3,3,CHN,China,5,5,12,2
3,4,4,AUS,Australia,5,4,9,0
4,5,5,KOR,South Korea,5,3,9,1
...,...,...,...,...,...,...,...,...
200,38,201,BVI,British Virgin Islands,0,0,0,0
201,38,202,AVI,American Virgin Islands,0,0,0,0
202,38,203,YEM,Yemen,0,0,0,0
203,38,204,ZAM,Zambia,0,0,0,0


In [23]:
df.columns

Index(['rank', 'sequence', 'code', 'name', 'gold', 'silver', 'total',
       'bronze'],
      dtype='object')

---

### 2. Athletes per country

We could get this data from a few places, such as the IOC. We'll use an article by CBS news and scrape the data from there.

In [2]:
import selenium as se
from selenium import webdriver
from bs4 import BeautifulSoup

import pandas as pd

In [3]:
url = 'https://www.cbsnews.com/news/athletes-olympics-numbers-2024-paris-games/'

# Use selenium to get the page source
driver = webdriver.Chrome()
driver.get(url)
html = driver.page_source
driver.quit()


# Use BeautifulSoup to parse the html
soup = BeautifulSoup(html, 'html.parser')

The list is contained within a `ul` tag.

In [4]:
# Find which ul tag contains the data we want
ul_list = soup.find_all('ul')
for i in range(len(ul_list)):
    print(i, ul_list[i].get_text(strip=True))

0 LatestU.S.WorldPoliticsEntertainmentHealthWatchMoneyWatchInvestigationsCrimeSpaceSportsEssentialsBrand StudioLocal NewsBaltimoreBay AreaBostonChicagoColoradoDetroitLos AngelesMiamiMinnesotaNew YorkPhiladelphiaPittsburghSacramentoTexasLiveCBS News 24/7BaltimoreBay AreaBostonChicagoColoradoDetroitLos AngelesMiamiMinnesotaNew YorkPhiladelphiaPittsburghSacramentoTexasShows48 Hours60 MinutesAmerica DecidesCBS Evening NewsCBS MorningsCBS News Eye on AmericaCBS News MorningsCBS ReportsCBS Saturday MorningThe Daily ReportThe DishFace the NationHere Comes the SunPerson to PersonSunday MorningThe TakeoutThe UpliftWeekenderCBS News InvestigatesPhotosPodcastsIn DepthNewslettersDownload Our AppCBS News TeamExecutive TeamParamount ShopParamount+Join Our Talent CommunityRSS FeedsA Moment With...Innovators & DisruptorsBrand Studio
1 U.S.WorldPoliticsEntertainmentHealthWatchMoneyWatchInvestigationsCrimeSpaceSportsEssentialsBrand Studio
2 BaltimoreBay AreaBostonChicagoColoradoDetroitLos AngelesMiamiMi

The data we want is in tag 8.

In [5]:
data = ul_list[8].find_all('li')
data

[<li>Afghanistan, 6</li>,
 <li>Albania, 8</li>,
 <li>Algeria, 45</li>,
 <li>American Samoa, 2</li>,
 <li>Andorra, 2</li>,
 <li>Angola, 24</li>,
 <li>Antigua and Barbuda, 5</li>,
 <li>Argentina, 136</li>,
 <li>Armenia, 15</li>,
 <li>Aruba, 6</li>,
 <li>Australia, 460</li>,
 <li>Austria, 81</li>,
 <li>Azerbaijan, 47</li>,
 <li>Bahamas, 18</li>,
 <li>Bahrain, 14</li>,
 <li>Bangladesh, 5</li>,
 <li>Barbados, 4</li>,
 <li>Belgium, 165</li>,
 <li>Belize, 1</li>,
 <li>Benin, 5</li>,
 <li>Bermuda, 8</li>,
 <li>Bhutan, 3</li>,
 <li>Bolivia, 4</li>,
 <li>Bosnia and Herzegovina, 5</li>,
 <li>Botswana, 13</li>,
 <li>Brazil, 277</li>,
 <li>Brunei Darussalam, 3</li>,
 <li>Bulgaria, 46</li>,
 <li>Burkina Faso, 8</li>,
 <li>Burundi, 7</li>,
 <li>Cabo Verde, 7</li>,
 <li>Cambodia, 3</li>,
 <li>Cameroon, 6</li>,
 <li>Canada, 316</li>,
 <li>Cayman Islands, 4</li>,
 <li>Central African Republic, 4</li>,
 <li>Chad, 3</li>,
 <li>Chile, 48</li>,
 <li>Chinese Taipei, 60</li>,
 <li>Colombia, 88</li>,
 <li>Como

In [6]:
# For each item in the list, split the text into two parts and store in a dictionary
data_dict = {}
for item in data:
    print(item.get_text(strip=True))
    # Add some custom rules to handle countries with commas in their names
    if 'Hong Kong' in item.get_text(strip=True):
        key, value = item.get_text(strip=True).split(', China, ')
    elif 'Virgin Islands, British' in item.get_text(strip=True):
        key = 'British Virgin Islands'
        value = 4
    elif 'Virgin Islands, US' in item.get_text(strip=True):
        key = 'US Virgin Islands'
        value = 5
    else:
        key, value = item.get_text(strip=True).split(', ')
    data_dict[key] = value

Afghanistan, 6
Albania, 8
Algeria, 45
American Samoa, 2
Andorra, 2
Angola, 24
Antigua and Barbuda, 5
Argentina, 136
Armenia, 15
Aruba, 6
Australia, 460
Austria, 81
Azerbaijan, 47
Bahamas, 18
Bahrain, 14
Bangladesh, 5
Barbados, 4
Belgium, 165
Belize, 1
Benin, 5
Bermuda, 8
Bhutan, 3
Bolivia, 4
Bosnia and Herzegovina, 5
Botswana, 13
Brazil, 277
Brunei Darussalam, 3
Bulgaria, 46
Burkina Faso, 8
Burundi, 7
Cabo Verde, 7
Cambodia, 3
Cameroon, 6
Canada, 316
Cayman Islands, 4
Central African Republic, 4
Chad, 3
Chile, 48
Chinese Taipei, 60
Colombia, 88
Comoros, 4
Congo, 4
Cook Islands, 2
Costa Rica, 6
Croatia, 73
Cuba, 61
Cyprus, 15
Czechia, 112
CÃ´te d'Ivoire, 13
Democratic People's Republic of Korea, 16
Democratic Republic of Timor-Leste, 4
Democratic Republic of the Congo, 6
Denmark, 123
Djibouti, 7
Dominica, 4
Dominican Republic, 57
Ecuador, 40
Egypt, 149
El Salvador, 8
Equatorial Guinea, 3
Eritrea, 13
Estonia, 24
Eswatini, 3
Ethiopia, 32
Federated States of Micronesia, 3
Fiji, 33
Finland,

In [7]:
data_dict

{'Afghanistan': '6',
 'Albania': '8',
 'Algeria': '45',
 'American Samoa': '2',
 'Andorra': '2',
 'Angola': '24',
 'Antigua and Barbuda': '5',
 'Argentina': '136',
 'Armenia': '15',
 'Aruba': '6',
 'Australia': '460',
 'Austria': '81',
 'Azerbaijan': '47',
 'Bahamas': '18',
 'Bahrain': '14',
 'Bangladesh': '5',
 'Barbados': '4',
 'Belgium': '165',
 'Belize': '1',
 'Benin': '5',
 'Bermuda': '8',
 'Bhutan': '3',
 'Bolivia': '4',
 'Bosnia and Herzegovina': '5',
 'Botswana': '13',
 'Brazil': '277',
 'Brunei Darussalam': '3',
 'Bulgaria': '46',
 'Burkina Faso': '8',
 'Burundi': '7',
 'Cabo Verde': '7',
 'Cambodia': '3',
 'Cameroon': '6',
 'Canada': '316',
 'Cayman Islands': '4',
 'Central African Republic': '4',
 'Chad': '3',
 'Chile': '48',
 'Chinese Taipei': '60',
 'Colombia': '88',
 'Comoros': '4',
 'Congo': '4',
 'Cook Islands': '2',
 'Costa Rica': '6',
 'Croatia': '73',
 'Cuba': '61',
 'Cyprus': '15',
 'Czechia': '112',
 "CÃ´te d'Ivoire": '13',
 "Democratic People's Republic of Korea":

In [10]:
# Create a dataframe from the dictionary
df = pd.DataFrame(data_dict.items(), columns=['country', 'athletes'])

# Import country converter as coco
import country_converter as coco
# Add a column with the ISO3 country codes
df['code'] = coco.convert(names=df['country'], to='ISO3')
df['iso2'] = coco.convert(names=df['country'], to='ISO2')

df = df[['iso2', 'code', 'country', 'athletes']]
df

Refugee Olympic Team not found in regex
More than one regular expression match for Individual Neutral Athletes from Russia and Belarus
More than one regular expression match for Individual Neutral Athletes from Russia and Belarus
More than one regular expression match for Individual Neutral Athletes from Russia and Belarus
More than one regular expression match for Individual Neutral Athletes from Russia and Belarus
More than one regular expression match for Individual Neutral Athletes from Russia and Belarus
More than one regular expression match for Individual Neutral Athletes from Russia and Belarus
More than one regular expression match for Individual Neutral Athletes from Russia and Belarus
More than one regular expression match for Individual Neutral Athletes from Russia and Belarus
More than one regular expression match for Individual Neutral Athletes from Russia and Belarus
More than one regular expression match for Individual Neutral Athletes from Russia and Belarus
More than 

,iso2,code,country,athletes
0,AF,AFG,Afghanistan,6
1,AL,ALB,Albania,8
2,DZ,DZA,Algeria,45
3,AS,ASM,American Samoa,2
4,AD,AND,Andorra,2
...,...,...,...,...
201,VI,VIR,US Virgin Islands,5
202,YE,YEM,Yemen,4
203,ZM,ZMB,Zambia,28
204,ZW,ZWE,Zimbabwe,7


In [11]:
df.to_csv('athletes.csv', index=False)

---

### 3. Population

We've saved World Population data previously.

In [85]:
path = '/Users/j.i.hellings/Documents/GrowthDay/1_DataDeepDive/data/C1_Population.csv'
df_pop = pd.read_csv(path)

# df_pop.to_csv('../../Data/UN_population_2023.csv', index=False)
df_pop


,Rank,Country,Population,Share,ISO3,Continent,P21
0,1,India,1428627663,0.1776,IND,Asia,True
1,2,China,1425671352,0.1772,CHN,Asia,True
2,3,United States,339996563,0.0423,USA,America,True
3,4,Indonesia,277534122,0.0345,IDN,Asia,True
4,5,Pakistan,240485658,0.0299,PAK,Asia,True
...,...,...,...,...,...,...,...
229,230,Montserrat,4386,0.0000,MSR,America,False
230,231,Falkland Islands,3791,0.0000,FLK,America,False
231,232,Niue,1935,0.0000,NIU,Oceania,False
232,233,Tokelau,1893,0.0000,TKL,Oceania,False


### 4. GDP

World Bank data:
- GDP, PPP, Current International $: `NY.GDP.MKTP.PP.CD`

**Method 1**: Use World Bank API to get data for each country as needed

In [66]:
def get_wb_data(iso3: str, code='NY.GDP.MKTP.PP.CD'):
    # Make request
    # url = f'https://api.worldbank.org/v2/country/{iso3}/indicator/{code}?format=json&per_page=3'
    url = f'https://api.worldbank.org/v2/country/{iso3}/indicator/{code}?format=json&strdate=2020-01-01&enddate=2023-12-31'
    response = requests.get(url)
    try:
        data = json.loads(response.text)
        i = 0
        value = data[1][i]['value']
        while not value:
            i += 1
            value = data[1][i]['value']
            # If no data is found, try the next 5 years
            if i > 5:
                break
        if not value:
            print(f'No data found for {iso3}')
    except:
        print(f'No data found for {iso3}')
        value = None
    return value
        

In [64]:
get_wb_data('AFG')

86077580286.4421

In [67]:
import requests, json

df = df_pop.copy()
df['GDP'] = None
for i, row in df.iterrows():
    value = get_wb_data(row['ISO3'])
    df.loc[i, 'GDP'] = value

df.head()

No data found for BRA
No data found for YEM
No data found for VEN
No data found for PRK
No data found for TWN
No data found for TCD
No data found for GTM
No data found for GIN
No data found for CUB
No data found for SSD
No data found for ERI
No data found for REU
No data found for ESH
No data found for GLP
No data found for MTQ
No data found for MYT
No data found for GUF
No data found for PYF
No data found for NCL
No data found for GUM
No data found for IMN
No data found for MNP
No data found for ASM
No data found for LIE
No data found for MCO
No data found for GIB
No data found for MAF
No data found for VGB
No data found for BES
No data found for COK
No data found for AIA
No data found for WLF
No data found for BLM
No data found for SPM
No data found for SHN
No data found for MSR
No data found for FLK
No data found for NIU
No data found for TKL
No data found for VAT


,Rank,Country,Population,Share,ISO3,Continent,P21,GDP
0,1,India,1428627663,0.1776,IND,Asia,True,14537383619971.599609
1,2,China,1425671352,0.1772,CHN,Asia,True,34643706848019.898438
2,3,United States,339996563,0.0423,USA,America,True,27360934999999.898438
3,4,Indonesia,277534122,0.0345,IDN,Asia,True,4333084496745.120117
4,5,Pakistan,240485658,0.0299,PAK,Asia,True,1493905948493.040039


**Method 2**: Use main dataset

Data downloaded as Excel file, so we'll need to clean it and extract the most recent value 
for each country.

In [74]:
df_gdp = pd.read_excel('../../Data/world-bank_gdp-ppp-current_2023.xls', sheet_name='Data', skiprows=3)

# Drop columns
df_gdp = df_gdp.drop(columns=['Indicator Name', 'Indicator Code'])

# Rename columns
df_gdp = df_gdp.rename(columns={'Country Name': 'country', 'Country Code': 'ISO3'})

# Rotate
df_gdp = df_gdp.melt(id_vars=['country', 'ISO3'], var_name='year', value_name='GDP')

# Now, we want to get the most recent GDP value for each country
#   Filter to 2019 or later
df_gdp = df_gdp[df_gdp['year'].astype(int) >= 2019].copy()

#   Drop `NaN` GDP values
df_gdp = df_gdp.dropna(subset=['GDP'])

#  Sort by year
df_gdp = df_gdp.sort_values(['country', 'year'])

# Drop duplicates, keeping the last value
df_gdp = df_gdp.drop_duplicates(subset='country', keep='last').reset_index(drop=True)
df_gdp

,country,ISO3,year,GDP
0,Afghanistan,AFG,2022,8.607758e+10
1,Africa Eastern and Southern,AFE,2023,3.296249e+12
2,Africa Western and Central,AFW,2023,2.709825e+12
3,Albania,ALB,2023,5.875096e+10
4,Algeria,DZA,2023,7.765398e+11
...,...,...,...,...
242,Virgin Islands (U.S.),VIR,2021,4.895223e+09
243,West Bank and Gaza,PSE,2023,3.041812e+10
244,World,WLD,2023,1.846537e+14
245,Zambia,ZMB,2023,8.486303e+10


In [25]:
df.head()

,Rank,Country,Population,Share,ISO3,Continent,P21,GDP
0,1,India,1428627663,0.1776,IND,Asia,True,14537383619971.599609
1,2,China,1425671352,0.1772,CHN,Asia,True,34643706848019.898438
2,3,United States,339996563,0.0423,USA,America,True,27360934999999.898438
3,4,Indonesia,277534122,0.0345,IDN,Asia,True,4333084496745.120117
4,5,Pakistan,240485658,0.0299,PAK,Asia,True,1493905948493.040039


In [27]:
# Add ISO2 codes
df['ISO2'] = coco.convert(names=df['ISO3'], to='ISO2')

### GDP per capita

In [97]:
df_gdp_pc = pd.read_excel('../../Data/world-bank_gdp-pc-ppp-current_2023.xls', sheet_name='Data', skiprows=3)

# Drop columns
df_gdp_pc = df_gdp_pc.drop(columns=['Indicator Name', 'Indicator Code'])

# Rename columns
df_gdp_pc = df_gdp_pc.rename(columns={'Country Name': 'country', 'Country Code': 'ISO3'})

# Rotate
df_gdp_pc = df_gdp_pc.melt(id_vars=['country', 'ISO3'], var_name='year', value_name='GDP-pc')

# Now, we want to get the most recent GDP value for each country
#   Filter to 2019 or later
df_gdp_pc = df_gdp_pc[df_gdp_pc['year'].astype(int) >= 2019].copy()

#   Drop `NaN` GDP values
df_gdp_pc = df_gdp_pc.dropna(subset=['GDP-pc'])

#  Sort by year
df_gdp_pc = df_gdp_pc.sort_values(['country', 'year'])

# Drop duplicates, keeping the last value
df_gdp_pc = df_gdp_pc.drop_duplicates(subset='country', keep='last').reset_index(drop=True)
df_gdp_pc

,country,ISO3,year,GDP-pc
0,Afghanistan,AFG,2022,2092.879952
1,Africa Eastern and Southern,AFE,2023,4459.764134
2,Africa Western and Central,AFW,2023,5389.580896
3,Albania,ALB,2023,21395.325037
4,Algeria,DZA,2023,17026.961675
...,...,...,...,...
242,Virgin Islands (U.S.),VIR,2021,46238.059654
243,West Bank and Gaza,PSE,2023,5888.394311
244,World,WLD,2023,23009.815026
245,Zambia,ZMB,2023,4125.625556


In [112]:
df_gdp[df_gdp['country'] == 'Kosovo']['ISO3'].values[0]

'XKX'

---

### 5. Combine data

**5.1** Merge population & GDP

In [83]:
df_pop.head()

,Rank,Country,Population,Share,ISO3,Continent,P21
0,1,India,1428627663,0.1776,IND,Asia,True
1,2,China,1425671352,0.1772,CHN,Asia,True
2,3,United States,339996563,0.0423,USA,America,True
3,4,Indonesia,277534122,0.0345,IDN,Asia,True
4,5,Pakistan,240485658,0.0299,PAK,Asia,True


In [99]:
df_main1 = pd.merge(df_pop, df_gdp[['ISO3', 'GDP']], on='ISO3', how='left')
df_main2 = pd.merge(df_main1, df_gdp_pc[['ISO3', 'GDP-pc']], on='ISO3', how='left')
df_main2

,Rank,Country,Population,Share,ISO3,Continent,P21,GDP,GDP-pc
0,1,India,1428627663,0.1776,IND,Asia,True,1.453738e+13,10175.767974
1,2,China,1425671352,0.1772,CHN,Asia,True,3.464371e+13,24557.638953
2,3,United States,339996563,0.0423,USA,America,True,2.736093e+13,81695.187071
3,4,Indonesia,277534122,0.0345,IDN,Asia,True,4.333084e+12,15612.799124
4,5,Pakistan,240485658,0.0299,PAK,Asia,True,1.493906e+12,6212.037595
...,...,...,...,...,...,...,...,...,...
229,230,Montserrat,4386,0.0000,MSR,America,False,NaN,NaN
230,231,Falkland Islands,3791,0.0000,FLK,America,False,NaN,NaN
231,232,Niue,1935,0.0000,NIU,Oceania,False,NaN,NaN
232,233,Tokelau,1893,0.0000,TKL,Oceania,False,NaN,NaN


**5.2** Merge with Paris Olympic athlete counts

In [100]:
df_athletes = pd.read_csv('athletes.csv')
df_athletes.head()

,iso2,code,country,athletes
0,AF,AFG,Afghanistan,6
1,AL,ALB,Albania,8
2,DZ,DZA,Algeria,45
3,AS,ASM,American Samoa,2
4,AD,AND,Andorra,2


In [103]:
df_athletes = pd.read_csv('athletes.csv')

df_main = pd.merge(df_athletes, df_main2, left_on='code', right_on='ISO3', how='left')

In [104]:
# Drop cols
df_main = df_main.drop(columns=['Country', 'ISO3'])
df_main

,iso2,code,country,athletes,Rank,Population,Share,Continent,P21,GDP,GDP-pc
0,AF,AFG,Afghanistan,6,36.0,42239854.0,0.0053,Asia,False,8.607758e+10,2092.879952
1,AL,ALB,Albania,8,138.0,2832439.0,0.0004,Europe,False,5.875096e+10,21395.325037
2,DZ,DZA,Algeria,45,34.0,45606480.0,0.0057,Africa,False,7.765398e+11,17026.961675
3,AS,ASM,American Samoa,2,212.0,43914.0,0.0000,Oceania,False,NaN,NaN
4,AD,AND,Andorra,2,202.0,80088.0,0.0000,Europe,False,5.733298e+09,71587.481776
...,...,...,...,...,...,...,...,...,...,...,...
201,VI,VIR,US Virgin Islands,5,199.0,98750.0,0.0000,America,False,4.895223e+09,46238.059654
202,YE,YEM,Yemen,4,44.0,34449825.0,0.0043,Asia,False,NaN,NaN
203,ZM,ZMB,Zambia,28,63.0,20569737.0,0.0026,Africa,False,8.486303e+10,4125.625556
204,ZW,ZWE,Zimbabwe,7,74.0,16665409.0,0.0021,Africa,False,6.499342e+10,3899.899233


In [105]:
# Replace 'not found' with missing
df_main = df_main.replace('not found', None)

Validate any countries with missing population

In [108]:
# Check for countries without population
df_main[df_main['Population'].isnull()]

,iso2,code,country,athletes,Rank,Population,Share,Continent,P21,GDP,GDP-pc
99,XK,XKX,Kosovo,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,None,None,Refugee Olympic Team,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205,"['BY', 'RU']","['BLR', 'RUS']",Individual Neutral Athletes from Russia and Be...,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN


So we're missing Kosovo, which we should be able to get data for. Our World Bank data includes the GDP and GDP PC for Kosovo, so we just need Population.

There are some UN estimates [here](https://data.un.org/Data.aspx?q=Kosovo&d=PopDiv&f=variableID%3a12%3bcrID%3a412)


In [122]:
# Get Kosovo values for GDP and GDP per capita
xkx_gdp = df_gdp[df_gdp['country'] == 'Kosovo']['GDP'].values[0]
xkx_gdp_pc = df_gdp_pc[df_gdp_pc['country'] == 'Kosovo']['GDP-pc'].values[0]
xkx_pop = 1663594

In [123]:
# Updae Kosovo values
df_main.loc[df_main['country'] == 'Kosovo', 'Population'] = xkx_pop
df_main.loc[df_main['country'] == 'Kosovo', 'GDP'] = xkx_gdp
df_main.loc[df_main['country'] == 'Kosovo', 'GDP-pc'] = xkx_gdp_pc

Save to CSV

### Add country flags

In [124]:
df_main.to_csv('olympics_country-stats.csv', index=False)

---

### 6. Merge stats data with live medals data

In [107]:
medals = 'https://eco-data-cache.s3.eu-west-2.amazonaws.com/olympics/live_results/medals.csv'

df_medals = pd.read_csv(medals)
df_medals

,rank,sequence,code,name,gold,silver,total,bronze
0,1,1,CHN,China,8,6,16,2
1,2,2,JPN,Japan,7,2,13,4
2,3,3,FRA,France,6,9,21,6
3,4,4,GB,Great Britain,6,6,17,5
4,5,5,AUS,Australia,6,4,13,3
...,...,...,...,...,...,...,...,...
200,46,201,BVI,British Virgin Islands,0,0,0,0
201,46,202,AVI,American Virgin Islands,0,0,0,0
202,46,203,YEM,Yemen,0,0,0,0
203,46,204,ZAM,Zambia,0,0,0,0


Task: merge with our main country stats data. We'll need to account for any differences in names or code, since some countries 

In [125]:
df_main

,iso2,code,country,athletes,Rank,Population,Share,Continent,P21,GDP,GDP-pc
0,AF,AFG,Afghanistan,6,36.0,42239854.0,0.0053,Asia,False,8.607758e+10,2092.879952
1,AL,ALB,Albania,8,138.0,2832439.0,0.0004,Europe,False,5.875096e+10,21395.325037
2,DZ,DZA,Algeria,45,34.0,45606480.0,0.0057,Africa,False,7.765398e+11,17026.961675
3,AS,ASM,American Samoa,2,212.0,43914.0,0.0000,Oceania,False,NaN,NaN
4,AD,AND,Andorra,2,202.0,80088.0,0.0000,Europe,False,5.733298e+09,71587.481776
...,...,...,...,...,...,...,...,...,...,...,...
201,VI,VIR,US Virgin Islands,5,199.0,98750.0,0.0000,America,False,4.895223e+09,46238.059654
202,YE,YEM,Yemen,4,44.0,34449825.0,0.0043,Asia,False,NaN,NaN
203,ZM,ZMB,Zambia,28,63.0,20569737.0,0.0026,Africa,False,8.486303e+10,4125.625556
204,ZW,ZWE,Zimbabwe,7,74.0,16665409.0,0.0021,Africa,False,6.499342e+10,3899.899233


In [ ]:
# Define function for matching df_medals to df_main
def match_medals(df_main, df_medals):
    # Add empty columns that we will populate
    df_medals['athletes'] = None
    df_medals['population'] = None
    df_medals['GDP'] = None
    df_medals['GDP-pc'] = None
    df_medals['continent'] = None
    
    for i, row in df_main.iterrows():
        if row['iso2'] in df_medals['iso2'].values:
            df_main.loc[i, 'medals'] = df_medals[df_medals['iso2'] == row['iso2']]['total'].values[0]
        else:
            df_main.loc[i, 'medals'] = 'not found'
    # Merge the two dataframes
    df = pd.merge(df_main, df_medals, left_on='iso2', right_on='iso2', how='left')
    # Drop the 'code' column
    df = df.drop(columns=['code'])
    # Replace 'not found' with missing
    df = df.replace('not found', None)
    return df